# Export open text field into Excel for easy coding

This set of scripts reads sentences (+sources) from one Excel file (datafile.xlsx), retrieves the already coded data from coding Excel file (text_codes.xlsx), updates/exports all unique sentence text fields to the coding Excel file (text_codes.xlsx), without touching the existing coding. <br><br>
The preformatted output Excel sheet is designed for easy use; with existing freeze panes, filtering and printing setup, the free text and entered codes can be ordered, filtered and checked conviniently. The formatting and the codes stay in the Excel sheet even when new data is added with the script below.<br><br>
Target is that one can code the text fields into categories for further processing, and possible retrieval for further analysis.<br><br>
<b>Input:</b>
- datafile.xlsx (sheet: rdntext, source) -- random texts with source information
- text_codes.xlsx (sheet 'coding': rdmtext, code01, code02; sheet 'metadata': list of unique sources)

<b>Output:</b>
- text_codes.xlsx (sheet 'coding': rdmtext, code01, code02; sheet 'metadata': list of unique sources); <br>
sheet 'coding' with filtering and freeze pain, no duplicates, page and printing setup

# Import libraries

In [1]:
import pandas as pd

# Read generated texts

In [2]:
'''Read the files'''
newfile = 'datafile.xlsx'
df1 = pd.read_excel(newfile, header=0)
df1.shape

(115, 2)

In [3]:
# glimpse of unique pacomms
df1[(df1.rdmtext.duplicated(keep=False))&(~df1.rdmtext.isnull())].sample(5)

,rdmtext,source
60,Takaa sekin annat et oikea jo tieda silta.,src01
104,Korjannut lipullaan ole tee toi vavahtaen jaa ...,scr05
92,Kun purjeeton pienoinen tuo aitisikin ota jai ...,scr03
99,Han jalasta leveana jaa silloin.,scr04
108,Kaataa heinia toista jos nyt.,scr05


# Read the existing Excel coding file

In [4]:
'''Read the file with all CRF data from CRF3'''
oldfile =  'text_codes.xlsx'
df2 = pd.read_excel(oldfile, 'coding',header=0)
df2.shape

(73, 3)

In [5]:
df2.sample(5)

,rdmtext,code01,code02
56,Ristiin jai tai voi kykenee pistaen jauhoja to...,ne,NaN
45,Vei puheenaihe mielellaan puutavaraa ole tuo p...,NaN,NaN
38,On hiljainen semmoisia ne raastavat ylpeammin ...,ne,NaN
7,Roisto toivoa kun takana littea lintua vasten ...,te,NaN
20,Kun tosissaan haaveensa oma ainaisena.,NaN,NaN


In [6]:
# amount and list of subjects
len(df1.source.unique()), df1.source.unique()

(5, array(['src01', 'src02', 'scr03', 'scr04', 'scr05'], dtype=object))

## Temp file

In [7]:
'''New dataset, with irrelevant open text fields ignored'''
srcs = [i for i in df1.source.unique()]
df_temp = df1.loc[(~df1.rdmtext.isin([i for i in df2.rdmtext]))&
                  (df1.source.isin(srcs))&
                  (~df1.rdmtext.isin(['Pappi lie koyha jos rikas yha hahah.']))]

In [8]:
df_temp.tail()

,rdmtext,source
106,Ei hanella ai he rantaan peralla te puhalsi.,scr05
107,Ja on lyhtyja ajaapas repeisi nokkaan on.,scr05
108,Kaataa heinia toista jos nyt.,scr05
109,Korjannut lipullaan ole tee toi vavahtaen jaa ...,scr05
110,Saaliin odottaa kai pitaisi tulevan eli kulloin.,scr05


# Combine existing and new data

In [9]:
# create new data frame
df_all = pd.DataFrame(columns=df2.columns)

In [10]:
# new data from pacomm field only
df_all.rdmtext = df_temp.rdmtext.dropna()

In [11]:
# merge existing coding data
df_all = pd.concat([df2, df_all], axis=0)

In [12]:
# check: unique rdmtext only, duplicates only, all
len(df_all[~df_all.rdmtext.duplicated(keep=False)]), len(df_all[df_all.rdmtext.duplicated(keep=False)]), len(df_all)

(73, 40, 113)

In [13]:
# check: unique rdmtext only
df_all[~df_all.rdmtext.duplicated(keep=False)].head(5)

,rdmtext,code01,code02
0,Me ja mukaan ai no matkaa nahnyt.,me,nähnyt
1,Nyt ukkovaari ehtymatta semmoista tassakaan ke...,ukkovaari,eli
2,Ne en ryypannyt tassakaan tervehtii.,ne,NaN
3,Semmoisia ihmisilla vielakaan te on seitsemia ...,te,NaN
4,Ja seka mita puhe on te.,te,NaN


In [14]:
# check: duplicate rdmtext only
df_all[df_all.rdmtext.duplicated(keep=False)].head(5)

,rdmtext,code01,code02
8,Iso ota mokottaa jos lekkerin poydalta kuuluvat.,NaN,NaN
9,Yha kahvipannu lahetakaan varmaankin valahtiva...,NaN,NaN
15,Ajautua voi ole iso pitkana jostain han.,NaN,NaN
18,Kulti liene hanet se jatti pilan jonka en.,NaN,NaN
25,Ai se ottaisit rukoilen kullakin saastaen.,NaN,NaN


In [15]:
# check: all data
df_all.sample(5)

,rdmtext,code01,code02
78,Ai se ottaisit rukoilen kullakin saastaen.,NaN,NaN
72,Seinaa ota kay hyvina paljon taivas hauska.,NaN,NaN
5,Ai ne oikeastaan rukoukseni ja me hyvastinsa.,ne,NaN
49,Takaa sekin annat et oikea jo tieda silta.,NaN,NaN
86,Et antautuen torpparit toivotaan asiakseen aja...,NaN,NaN


In [16]:
'''Refs for the used functionalities'''
# http://xlsxwriter.readthedocs.io/example_pandas_column_formats.html
# https://xlsxwriter.readthedocs.io/working_with_autofilters.html
# http://xlsxwriter.readthedocs.io/example_panes.html
# http://xlsxwriter.readthedocs.io/page_setup.html
# http://xlsxwriterlua.readthedocs.io/example_protection.html #not used in this version

'''Data and filename'''
# data = df1[~df1.rdmtext.duplicated(keep=False)] # first time save
data = df_all[~df_all.rdmtext.duplicated(keep=False)] # drop duplicate open text fields
metadata = pd.DataFrame(df1.source.unique()) # list of unique subject ids
mysheet = 'coding'
metasheet = 'metadata'
save_filename = 'text_codes.xlsx'

(rows, cols) = data.shape #for column formatting


writer = pd.ExcelWriter(save_filename)
workbook  = writer.book

'''Sheet definition, needs to be in this order to get these in the same order in the Excel sheet'''
#coding data
data.to_excel(writer,sheet_name=mysheet,index=False)
#metadata
metadata.to_excel(writer,sheet_name=metasheet,index=False)

#setup the coding sheet print area, formatting etc.
worksheet = writer.sheets[mysheet]

'''page setup for printing'''

worksheet.set_landscape()
worksheet.repeat_rows(0)
worksheet.print_area(0, 0, rows, cols-1)
worksheet.fit_to_pages(1, 0)

'''header formatting for printing'''

header = '&LPage &P of &N' + '&CFilename: &F' + '&RSheetname: &A'
footer = '&LCurrent date: &D' + '&RCurrent time: &T'
worksheet.set_header(header)
worksheet.set_footer(footer)

'''column formatting'''

wrap_format = workbook.add_format({'text_wrap': True, 'valign': 'top'})

worksheet.set_column('A:A', 75, wrap_format) # rdmtext (cols, width, format), wrap when needed
worksheet.set_column('B:C', 17) # pain_status--device_comm
# this could be done also column by column:
# worksheet.set_column('C:C', 17) # subj_status
# worksheet.set_column('D:D', 17) # nurse_activity
# worksheet.set_column('E:E', 17) # device_status
# worksheet.set_column('F:F', 17) # device_comm

'''row formatting'''

worksheet.freeze_panes(1, 0) #freeze panes
worksheet.autofilter(0, 0, rows, cols-1)  # Set filtering to the worksheet (startrow, startcol, endrow, endcol)


writer.save()

In [17]:
# just a check
df1.loc[df1.rdmtext=='Takaa sekin annat et oikea jo tieda silta.']

,rdmtext,source
60,Takaa sekin annat et oikea jo tieda silta.,src01
89,Takaa sekin annat et oikea jo tieda silta.,scr03


In [18]:
# any nans?
df1.loc[df1.source=='nan']

,rdmtext,source


# Versions used

In [19]:
!python -V

Python 3.6.3 :: Anaconda, Inc.


In [20]:
print ('pandas',pd._version.get_versions()['version'])

pandas 0.20.3
